In [1]:
from pycalphad import Database, Model, variables as v

dbf = Database('alfe_sei.TDB')

mod = Model(dbf, ['AL', 'FE', 'VA'], 'B2_BCC')

In [2]:
from collections import OrderedDict
from pycalphad.core.constraints import build_constraints
res = build_constraints(mod, [v.P, v.T] + mod.site_fractions, OrderedDict([(v.P, 101325), (v.T, 300), (v.N, 1), (v.X('FE'), 0.2)]))

In [3]:
import numpy as np
res.multiphase_jac.kernel(np.array([[101325, 300, 0.5, 0.5, 0.5, 0.5, 1, 1]]), np.array([]))

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  , -0.25,  0.25, -0.25,  0.25,  0.  ,  0.5 ]])

In [10]:
from pycalphad import equilibrium

res = equilibrium(dbf, ['AL', 'FE', 'VA'], 'B2_BCC', {v.P: 101325, v.T: 300, v.N: 1, v.X('FE'): 0.5}, verbose=True)

Components: AL FE VA
Phases: B2_BCC [done]
('Redundant phase:', CompositionSet(B2_BCC, [ 0.81437227  0.18562773], NP=1e-06, GM=-18380.583711649928))
Removing CompositionSet(B2_BCC, [ 0.81437227  0.18562773], NP=nan, GM=-18380.583711649928)
('x', array([  1.01325000e+05,   3.00000000e+02,   4.30861723e-01,
         5.69138277e-01,   5.69138277e-01,   4.30861723e-01,
         1.00000000e+00,   1.00000100e+00]))
('constraint_jac', array([[ 1.       ,  1.       ,  0.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  1.       ,  1.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  1.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,  1.       ],
       [-0.2500005,  0.2500005, -0.2500005,  0.2500005,  0.       ,  0.5      ]]))
('x', array([  1.01325000e+05,   3.00000000e+02,   4.30861723e-01,
         5.69138277e-01,   5.69138277e-01,   4.30861723e-01,
         1.00000000e+00,   1.0000

In [5]:
res

<xarray.Dataset>
Dimensions:    (N: 1, P: 1, T: 1, X_FE: 1, component: 2, internal_dof: 5, vertex: 2)
Coordinates:
  * P          (P) float64 1.013e+05
  * N          (N) float64 1.0
  * component  (component) <U2 'AL' 'FE'
  * vertex     (vertex) int64 0 1
  * T          (T) float64 300.0
  * X_FE       (X_FE) float64 0.2
Dimensions without coordinates: internal_dof
Data variables:
    GM         (N, P, T, X_FE) float64 nan
    Y          (N, P, T, X_FE, vertex, internal_dof) float64 nan nan nan nan ...
    MU         (N, P, T, X_FE, component) float64 nan nan
    X          (N, P, T, X_FE, vertex, component) float64 nan nan nan nan
    NP         (N, P, T, X_FE, vertex) float64 nan nan
    Phase      (N, P, T, X_FE, vertex) <U6 '' ''
Attributes:
    engine:   pycalphad 0.7+13.ga458f2c3.dirty
    created:  2018-04-18T04:20:07.773429

In [9]:
mod.get_multiphase_constraint_contribution(v.N).diff(v.Y('B2_BCC', 1, 'FE'))

-0.5*(0.5*B2_BCC0AL + 0.5*B2_BCC1AL)/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)**2 - 0.5*(0.5*B2_BCC0FE + 0.5*B2_BCC1FE)/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)**2 + 0.5/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)